# I. Finetuning PierreMaxime/llama-3-8b-chat-doctor

### 1. Import des modules et connexion HF et Wandb

In [ ]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [ ]:
# Import des différents modules et bibliothèques 
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [ ]:
# Connexion à Hugging Face et à Wandb
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import wandb

!git config --global credential.helper store
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF")

login(token=hf_token, add_to_git_credential=True)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)

run = wandb.init(
    project='Fine-tune Llama 3 8B on Medical Dataset', 
    job_type="training", 
    anonymous="allow"
)


### 2. Importation du dataset

In [ ]:
# Introduire le modèle de base + Dataset de training + nom du nouveau modèle
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model = "llama-3-8b-chat-doctor"

In [ ]:
# Importation du dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(2000))

### 3. Preprocessing 

In [ ]:
# Définition du format des données (tout mettre dans une colonne texte)
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Application de la fonction de formatage 
dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

len(dataset)

In [ ]:
# Définir le nombre de données qui seront utilisées pour le test 
dataset = dataset.train_test_split(test_size=0.1)

### 4. Modification de Llama3

In [ ]:
torch_dtype = torch.float16 # Définition du type de données utilisé par PyTorch pour les calculs tensoriels
attn_implementation = "eager" # Opération excécutée immédiatement (top pour le jupyter notebook) ≠ "graph" (mieux pour des gros modèles)

In [ ]:
# Configuration de QLoRA (méthode de quantification : Quantized Low Rank Adapter), obligatoire car on a une contrainte de mémoire
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Chargement du modèle
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

In [ ]:
# Chargement du tokenizer (ChatML template qui distingue l'user de l'assistant)
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

### 5. Training

In [ ]:
# Configuration de LoRA, paramètres pour améliorer le temps d'entrainement 
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
# Définir les arguments de training 
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=4,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [ ]:
# Configuration du processus d'entraînement 
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

trainer.train()

In [ ]:
wandb.finish()
model.config.use_cache = True

### 6. Inférence sur le modèle non merge avec llama3

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Hello doctor, I have bad headache. How do I get rid of it?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

### 7. Enregistrement du modèle finetune

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)